# Mitiq Tutorial

Before running this tutorial, make sure you have `mitiq` installed.

```bash
pip install mitiq
```

Use `pip list | grep mitiq` to ensure it is installed.

# Goals

- learn how to apply ZNE on a basic workflow
- apply both ZNE and DDD in conjunction


In [ ]:
# if you're using GitHub codespaces, you can use this to install directly in the notebook
%pip install mitiq

## Executors

Executors are python functions that consume a quantum circuit, and output an expectation value.
A type signature might look something like `Circuit -> float`.
That said, executors can have additional arguments used to control other parts of the execution process.
The circuit must be the first argument, however.


In [ ]:
import cirq


def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""

    # add depolarizing noise

    return (
        cirq.DensityMatrixSimulator().simulate(circuit).final_density_matrix[0, 0].real
    )

## ZNE

First, we define a simple circuit to work with.


In [ ]:
a, b, c = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    [
        cirq.H(a),
        cirq.CNOT(a, b),
        cirq.CNOT(b, c),
        cirq.S(a),
    ]
)

In [ ]:
print(circuit)

### Unitary Folding

Making the circuit longer, and hence noisier.


In [ ]:
from mitiq.zne.scaling import fold_gates_at_random, fold_global


folded_circuit = "..."

print(folded_circuit)

### Extrapolation

Computing the Zero-Noise limit.


In [ ]:
from mitiq.zne.inference import RichardsonFactory, LinearFactory, ExpFactory

factory = ...

factory.run(circuit, execute)
factory.reduce()

In [ ]:
factory.plot_fit();

Randomized Benchmarking circuits are circuits that are in effect, equivalent to the identity. Hence, the ideal probability that the end state is $|00\cdots 0\rangle$ is 1.


In [ ]:
from mitiq.benchmarks import generate_rb_circuits

circuit = generate_rb_circuits(2, num_cliffords=20)[0]

# print(circuit)

In [ ]:
from mitiq import zne


true_value = execute(circuit, noise_level=0.0)
noisy_value = execute(circuit)
zne_value = zne.execute_with_zne(circuit, execute)

print(f"Error w/o  Mitiq: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w Mitiq:    {abs((true_value - zne_value) / true_value):.3f}")

### Exercise

Keeping the circuit the same, can you get a smaller error by modifying some of the ZNE options?


In [ ]:
from mitiq.zne.scaling import fold_global, fold_gates_at_random, identity_insertion
from mitiq.zne.inference import (
    LinearFactory,
    RichardsonFactory,
    PolyFactory,
    ExpFactory,
)

inference_method = ...  # this will include scale factors, which you can play with!
noise_scaling_method = ...

zne_value = zne.execute_with_zne(
    circuit,
    execute,
    factory=inference_method,
    scale_noise=noise_scaling_method,
)

print(f"Error w/o  Mitiq: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w Mitiq:    {abs((true_value - zne_value) / true_value):.3f}")

## DDD


In [ ]:
from mitiq.benchmarks import generate_ghz_circuit

num_qubits = 6
ghz = generate_ghz_circuit(num_qubits)

print(ghz)

In [ ]:
from mitiq import ddd

ddd.execute_with_ddd(ghz, execute)